In [ ]:
pip install 'requests'

In [2]:
import requests
import pandas as pd
import numpy as np

To be able to use the Pôle Emploi API, we need to generate an access token with the client identifier and the secret key provided beforehand, which expires after 24 minutes. 
Once we have the access token, we can query the API and retrieve the job offers in JSON format.
Étape 1: Demander un 'access token' à Pôle Emploi Access Management en spécifiant les API souhaitées et l'id et secret.
    Pôle emploi access management génère et transmet un 'access token' valable pendant 24 minutes.
Étape 2: requêter une API: on fourni l'access token à chaque interrogation d'une API

Étape 1:

In [125]:
# URL pour obtenir le jeton d'accès
token_url = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire'

# Remplacez 'VOTRE_IDENTIFIANT_CLIENT' et 'VOTRE_CLE_SECRETE' par les valeurs réelles fournies par Pôle Emploi
client_id = 'PAR_poleemploijobsscrapin_4280e6316989cfda7bc759c90500ef347cc95780c6b1d5b67cca3db6c6d373d1'
client_secret = '70d252725959266d147615699b09d63746184cec836d7ff41f1bec6427215373'
# Scopes autorisés pour votre application
scopes = 'api_offresdemploiv2 o2dsoffre'  # Remplacez par les scopes valides pour votre application

# Corps de la requête avec ces paramètres
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': scopes
}

# En-tête de la requête
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Faire la requête pour obtenir le jeton d'accès
# Demander l'access token avec 'post'
response = requests.post(token_url, data=token_params, headers=headers)
access_token=response.json().get('access_token')


Étape 2:

In [126]:
# Requêter l'API avec 'get':
# Soit on spécifie les paramètres dans l'url soit on les mets dans une liste 'params'
#url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?range=0-2&sort=1&motsCles=data%20engineer'
url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search'

headers = {
    'Authorization': f'Bearer {access_token}'
}

# Paramètres de la requête
params = {
    'range': '0-137',
    'sort': '1',
    'motsCles': 'data engineer',
}

# Faire la requête avec les en-têtes d'authentification
reponse = requests.get(url, headers=headers, params=params)

# Afficher la réponse
print(reponse.status_code)
print(reponse.headers)
print(reponse.json())

200
{'Accept-Range': '150', 'Access-Control-Expose-Headers': 'Content-Range', 'Content-Range': 'offres 0-137/138', 'Date': 'Wed, 27 Dec 2023 20:01:40 GMT', 'Content-Type': 'application/json', 'Strict-Transport-Security': 'max-age=64281600; includeSubDomains', 'X-Frame-Options': 'ALLOW-FROM https://*.pole-emploi.fr', 'Set-Cookie': 'BIGipServerVS_EX035-VIPA-A4PMEX_HTTP.app~POOL_EX035-VIPA-A4PMEX_HTTP=3707111946.10062.0000; Path=/; HttpOnly, TS01585e85=01b3abf0a2344ecabb137ff7fef838603d3ca4dd15539ff9b2eeb91ee5576a2bdf6f78b027ca5967a1b10aab1f58e1203ac6aaddb9; Path=/; Domain=.api.pole-emploi.io; Secure; HTTPOnly', 'Transfer-Encoding': 'chunked'}
{'resultats': [{'id': '8743119', 'intitule': 'DATA ENGINEER (H/F)', 'description': 'LE BRIEF\nCDI -\xa0Data Engineer (H/F) - Nantes -\xa0 50K-55K - Télétravail partiel - ESN & Editeur de logiciels\nSolenn dAdsearch,\xa0Consultante spécialisée\xa0IT\xa0(Infrastructure, Data, Sécurité),\xa0recrute pour lun de ses partenaires, ESN et Editeur de logicie

In [127]:
#Sauvegarder les données dans un fichier json:
with open("pole_emploi_data.json", "w", encoding="utf-8") as json_file:
    json_file.write(reponse.text)

In [ ]:
#Convertir les données json en DataFrame:
data = reponse.json()
df2 = pd.json_normalize(data['resultats'])
df2.head()

In [ ]:
df2.info()

In [130]:
df2['experienceOuiNon'] = np.where(df2['experienceExige'] == 'D', 'non', np.where(df2['experienceExige'].isin(['E', 'S']), 'oui', None))


In [131]:
# Division de la colonne "lieuTravail.libelle" en "Dept" et "Ville"
df_Dept_ville = df2["lieuTravail.libelle"].str.split(" - ", expand=True)
df_Dept_ville.columns = ["Dept", "Ville"]

# Création des nouvelles colonnes "Departement" et "Ville" dans le dataframe
df2["lieuTravail.departement"] = df_Dept_ville["Dept"]
df2["lieuTravail.ville"] = df_Dept_ville["Ville"]

In [132]:
#Garder que les colonnes nécessaires
colonnes_conserver = ['id', 'intitule', 'description', 'dateActualisation', 'typeContrat', 'experienceExige','experienceOuiNon','lieuTravail.libelle', 'lieuTravail.latitude', 'lieuTravail.longitude', 'lieuTravail.codePostal', 'lieuTravail.commune','lieuTravail.departement','lieuTravail.ville', 'entreprise.description', 'entreprise.nom', 'salaire.libelle', 'salaire.commentaire', 'secteurActiviteLibelle', 'qualificationLibelle']  
df = df2.drop(columns=df2.columns.difference(colonnes_conserver))

In [ ]:
df.info()